In [879]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [880]:
import numpy as np

from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

In [881]:
#def get_v_values(p_pt,i):
#    '''
#    This function calculates the molar volume v values for
#    every p_pt and gas i, by solving the Van der Waals (molar)
#    equation of state for v.
#    
#    $(p+\frac{a}{v^2})(v-b)=RT$
#    
#    Where T=T_pt is fixed as 273.16K and a and b are fixed parameters for
#    each gas
#    
#    Inputs:
#    p_pt: a 1D array of the p_pt values (in kPa) on which we are calculating v
#    i: integer index to characterize a gas
#    
#    Returns:
#    v_values: a 1D array of the v values corresponding to each p_pt
#    '''
#    a = a_data[i]
#    b = b_data[i]
#    v_values=[]
#    for p in p_pt:
#        poly = np.poly1d([p, -R*T_pt -p*b, a, -a*b])
#        solutions = np.roots(poly) #This statement uses numpy roots to solve third order polynomial
#        for sol in solutions: #This control seq. select the only real solution.
#            if abs(sol.imag) < 1e-5:
#                v_values.append(sol.real)
#    return v_values

In [882]:
#def get_p_values(v_values,T,i):
#    '''
#    This functions calculates the p values for
#    every v in v_values, temperature T and gas i by
#    applying the van der waals equation of state.
#    
#    Inputs:
#    v_values: a 1d array of the v values of each measurement
#    T: float temperature (in K) for which we are calculating pressure
#    i: integer index to characterize a gas
#    
#    Returns:
#    p: a 1D array of the p values corresponding to gas i for each value of v
#    '''
#    a = a_data[i]
#    b = b_data[i]
#    p = []
#    for v in v_values:
#        p.append(R*T/(v-b)-a/v**2)
#    return p

In [883]:
#def get_empirical_temperature(p_pt,T,n_of_gases):
#    '''
#    This functions calculates the empirical temperature of thermal state T
#    for which p_pt and p have been previously calculated, by applying the 
#    definiton of empirical temperature.
#    
#    Inputs:
#    p_pt: a 1D array of the p_pt values for which we are calculating Theta
#    T: float temperature (in K) to measure
#    n_of_gases: number of gases to consider in the plot.
#    
#    Returns:
#    Theta: a 2D array of the values of empirical temperature for each gas (first index)
#    and for each value of p_pt (second index).
#    '''
#    
#    Theta = []
#    for i in range(n_of_gases):
#        v_values = get_v_values(p_pt,i)
#        p = get_p_values(v_values,T,i)
#        Theta.append(p/p_pt * 273.16) 
#    return Theta

In [884]:
#def update_figure(change):
#    '''
#    This function updates the plot figure whenever the T slider
#    value is changed or checkbox are turned on/off
#    
#    WARNING: Funtzio honek checkbox-a klikatzen duen bakoitzan Theta
#    berkalkulatzen du. Hori ez da oso efizientea. Oraingoz ez dut beste
#    modurik.
#    '''
#    obj = change.owner
#    
#    op=[0.0,0.0,0.0,0.0,0.0,0.0]
#    
#    if Hidrogen_check_121_003.value == True:
#        op[0] = 1.0
#    if Helium_check_121_003.value == True:
#        op[1] = 1.0
#    if Nitrogen_check_121_003.value == True:
#        op[2] = 1.0
#    if Oxygen_check_121_003.value == True:
#        op[3] = 1.0
#    if Neon_check_121_003.value == True:
#        op[4] = 1.0
#    if Argon_check_121_003.value == True:
#        op[5] = 1.0
#    
#    T = T_slider_121_002.value
#    y_values = get_empirical_temperature(p_pt,T,c)
#    scale_y = bqs.LinearScale(min = T-0.2, max = T+0.2)
#    
#    axis_y.scale=scale_y
#    data.y = y_values
#    data.scales = {'x': scale_x, 'y': scale_y}
#    data.opacities = op
#

In [885]:
def get_v(p_pt,T,gas):
    a = a_data[gas]
    b = b_data[gas]
    poly = np.poly1d([p_pt, -R*T_pt -p_pt*b, a, -a*b])
    solutions = np.roots(poly) #This statement uses numpy roots to solve third order polynomial equation
    for sol in solutions: #This control statement selects the only real solution.
            if abs(sol.imag) < 1e-5:
                v = sol.real
    return v

In [886]:
def get_Theta(p_pt,T,gas):
    
    a = a_data[gas]
    b = b_data[gas]
    v = get_v(p_pt,T,gas)
    p = (R*T/(v-b)-a/v**2)
    Theta = (p/p_pt * 273.16)
    return Theta

In [887]:
def update_current_point(change):
    gas = Gas_dropdown.value
    p_pt = p_pt_slider.value
    
    Theta = get_Theta(p_pt,T,gas)
    current_point.x = [p_pt]
    current_point.y = [Theta]
    current_point.colors = [colors[gas]]

In [888]:
def save_point(change):
    
    gas = Gas_dropdown.value
    if gas == 0:
        saved_points_0.x = np.append(saved_points_0.x, current_point.x )
        saved_points_0.y = np.append(saved_points_0.y, current_point.y )
    elif gas == 1:
        saved_points_1.x = np.append(saved_points_1.x, current_point.x)
        saved_points_1.y = np.append(saved_points_1.y, current_point.y)
    elif gas == 2:
        saved_points_2.x = np.append(saved_points_2.x, current_point.x)
        saved_points_2.y = np.append(saved_points_2.y, current_point.y)

In [889]:
def regression(change):
    gas = Gas_dropdown.value
    p_pt_values= np.linspace(1,120,n_of_points)
    
    Theta_values = []
    for i in range(n_of_points):
        Theta_values.append(get_Theta(p_pt,T,gas))
    lines.x = p_pt_values

In [890]:
def reset(change):
    
    saved_points_0.x = []
    saved_points_0.y = []
    saved_points_1.x = []
    saved_points_1.y = []
    saved_points_2.x = []
    saved_points_2.y = []

In [891]:
def f(change):
    return None

In [892]:
def f2():
    return None

In [893]:
## Fixed Parameters
#T_pt = 273.16 # Temperature of fixed point (triple point of water) in K. 
#R = 8.31446 # Gas constant in L*kPa/K/mol
n_of_points = 30 # Integer parameter to control how many points are to be calculated.

## Gas data
#labels = ["Hidrogenoa","Helioa","Nitrogenoa", "Oxigenoa", "Neona", "Argona"]
#latex = ["$H_2$", "$He$", "$N_2$", "$O_2$", "$Ne$", "$Ar$"]
#a_data = [24.7100, 3.4600, 137.00, 138.2, 21.3500, 135.500]
#b_data = [0.02661, 0.0238, 0.0387, 0.03186, 0.01709, 0.03201]
#colors = ['#0079c4','#f09205','#21c400','#dd4e4f', '#ae8ccd', '#86564b']
#c=6 #Cutoff: number of gases to consider
#opacities = [1.0, 1.0, 1.0, 0.0, 0.0, 0.0]

#Initial state:
#p_pt_values = np.linspace(1,120,n_of_points)
#T = 373.16
#Theta = get_empirical_temperature(p_pt, T, c)


# Fixed Parameters:
R = 8.31446 # Gas constant in L*kPa/K/mol
T_pt = 273.16 # Temperature of fixed point (triple point of water) in K.

# Gas data:
labels = ["Hidrogenoa","Helioa","Nitrogenoa", "Oxigenoa", "Neona", "Argona"]
latex = ["$H_2$", "$He$", "$N_2$", "$O_2$", "$Ne$", "$Ar$"]
a_data = [24.7100, 3.4600, 137.00, 138.2, 21.3500, 135.500]
b_data = [0.02661, 0.0238, 0.0387, 0.03186, 0.01709, 0.03201]
colors = ['#0079c4','#f09205','#21c400','#dd4e4f', '#ae8ccd', '#86564b']
c=6 #Cutoff: number of gases to consider
opacities = [1.0, 1.0, 1.0, 0.0, 0.0, 0.0]


# Initial state:
gas=0
p_pt = 100.0
T = 373.16
Theta = get_Theta(p_pt,T,gas)



#######################################
#######CREATE THE FIGURES##############
#######################################


fig_121_001 = bq.Figure(title='Tenperatura enpirikoak',
                marks=[],
                axes=[],
                animation_duration=0,
                legend_location='top-left',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=70, bottom=60, left=80, right=30),
                layout = widgets.Layout(width='100%'),
                toolbar = True,
    )

scale_x = bqs.LinearScale(min = 0.0, max = 150.0)
scale_y = bqs.LinearScale(min = T-0.2, max = T+0.2)

axis_x = bqa.Axis(scale=scale_x,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                #tick_values = np.linspace(0, max(p_pt), 7),
                num_ticks=5,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='p_pt (kPa)',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y = bqa.Axis(
                scale=scale_y,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='Theta (K)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_121_001.axes = [axis_x,axis_y]





#######################################
#########CREATE THE MARKS##############
#######################################


x_values = [ p_pt for i in range(c)]
y_values = Theta


current_point = bqm.Scatter(
                x = [p_pt], 
                y = [Theta], 
                scales = {'x': scale_x, 'y': scale_y}, 
                opacities = [1.0],
                visible = True, #True, #t == '1.00',
                colors = [colors[gas]],
                labels = [labels[gas]],
                display_legend = False
)


saved_points_0 = bqm.Scatter(
                x = [], 
                y = [], 
                scales = {'x': scale_x, 'y': scale_y}, 
                opacities = [1.0],
                visible = True, #True, #t == '1.00',
                colors = [colors[0]],
                labels = [labels[0]],
                display_legend = True
)

saved_points_1 = bqm.Scatter(
                x = [], 
                y = [], 
                scales = {'x': scale_x, 'y': scale_y}, 
                opacities = [1.0],
                visible = True, #True, #t == '1.00',
                colors = [colors[1]],
                labels = [labels[1]],
                display_legend = True
)

saved_points_2 = bqm.Scatter(
                x = [], 
                y = [], 
                scales = {'x': scale_x, 'y': scale_y}, 
                opacities = [1.0],
                visible = True, #True, #t == '1.00',
                colors = [colors[2]],
                labels = [labels[2]],
                display_legend = True
)

lines = bqm.Lines(
                x = [], 
                y = [[],[],[]], 
                scales = {'x': scale_x, 'y': scale_y}, 
                opacities = opacities,
                visible = True, #True, #t == '1.00',
                colors = colors,
                labels = labels,
                display_legend = False
)

fig_121_001.marks = [current_point, saved_points_0, saved_points_1, saved_points_2, lines]





#######################################
#########CONTROL WIDGETS###############
#######################################

## Left Block (temperature select)

T_slider = widgets.FloatSlider(
    min=100.0,
    max=400.0,
    step=0.1,
    value=T,
    description='T (ideal gas scale)',
    disabled=False,
    continuous_update=True,
    orientation='vertical',
    readout=True,
)

T_slider.observe(f,'value')


## Centre Block (gas and pressure select)

Gas_dropdown = widgets.Dropdown(
    options=[('Hidrogenoa',0), ('Helioa',1), ('Nitrogenoa',2)],
    value=0,
    description='Gasa:',
    disabled=False,
    layout = widgets.Layout(width='100%'),
)

Gas_dropdown.observe(update_current_point, 'value') 


p_pt_slider = widgets.FloatSlider(
    min=0.1,
    max=140.0,
    step=0.1,
    value=100.0,
    description='p_pt',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    layout = widgets.Layout(width='100%'),
    readout=True,
)

p_pt_slider.observe(update_current_point,'value')


save_button = widgets.Button(
    description='Gorde puntua',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me'
)

save_button.on_click(save_point)


regression_button = widgets.Button(
    description='Marraztu lerroa',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me'
)

regression_button.on_click(regression)


reset_button = widgets.Button(
    description='Ezabatu puntuak',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me'
)

reset_button.on_click(reset)


#######################################
############  LAYOUT  #################
#######################################


left_block_121_000 = widgets.VBox([], layout=widgets.Layout(width='10%', align_items='center'))
left_block_121_000.children = [T_slider]

centre_block_121_000 = widgets.VBox([], layout=widgets.Layout(width='20%', align_items='center'))
centre_block_121_000.children = [Gas_dropdown, p_pt_slider, save_button, regression_button, reset_button]


right_block_121_000 = widgets.VBox([], layout=widgets.Layout(width='70%', align_items='center'))

right_block_121_000.children = [fig_121_001]




main_block_121_000 = widgets.HBox([], layout=widgets.Layout(width='100%', align_self='center', align_items='center'))
main_block_121_000.children = [left_block_121_000, centre_block_121_000, right_block_121_000]
main_block_121_000

In [894]:
#Hobetzeko gauzak
# 1. update_figure funtzioa optimizatu Theta ez kalkulatzeko checkbox bat zapaltzen den bakoitzean
# 2. Gasen koloreak ondo aukeratu (O2, Ne eta Ar batez ere).
# 3. Gas batzuen kurba ia identikoa da. Beste batzuk aukeratu?
# 4. Layouta txukundu. Left block-a oso urruti dago, nola gerturatu?
# 5. LaTeX inplementatzeko modurik?